In [ ]:
import pandas as pd
import sys
test_json = pd.read_json("/content/drive/MyDrive/tqa_v1_train.json")

cols = ["text","question","answer"]
data_list=[]
for index,row in list(test_json.iterrows()):
  try:
    text = row['adjunctTopics']['Introduction']['content']['text'] + row['adjunctTopics']['Lesson Summary']['content']['text']
    questions = row['questions']['nonDiagramQuestions']
    for qid, qs in list(questions.items()):
        temp=[]
        question = qs['beingAsked']['processedText']
        answer = qs['answerChoices'][qs['correctAnswer']['processedText']]['processedText']
        temp.append(text)
        temp.append(question)
        temp.append(answer)
        data_list.append(temp)
  except:
    continue


In [2]:
import pandas as pd
new_df = pd.DataFrame(data_list, columns=cols)
new_df.to_csv("TQA_data.csv", index=False)

NameError: ignored

In [1]:
import pandas as pd
new_df=pd.read_csv('/content/drive/MyDrive/CNN-TQA/TQA_data.csv')
len(new_df)

4894

In [2]:
!pip install transformers
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [4]:
random_num = np.random.randint(0,len(new_df))
question = new_df["question"][random_num]
text = new_df["text"][random_num]

In [5]:
input_ids = tokenizer.encode(question, text)
print("The input has a total of {} tokens.".format(len(input_ids)))

The input has a total of 237 tokens.


In [6]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
for token, id in zip(tokens, input_ids):
    print('{:8}{:8,}'.format(token,id))

[CLS]        101
how        2,129
many       2,116
groups     2,967
are        2,024
there      2,045
in         1,999
men        2,273
##del      9,247
##ee       4,402
##vs      15,088
periodic  15,861
table      2,795
?          1,029
[SEP]        102
scientists   6,529
first      2,034
started    2,318
looking    2,559
for        2,005
a          1,037
way        2,126
to         2,000
organize  10,939
the        1,996
elements   3,787
in         1,999
the        1,996
1700      16,601
##s        2,015
.          1,012
they       2,027
were       2,020
trying     2,667
to         2,000
find       2,424
a          1,037
method     4,118
to         2,000
group      2,177
together   2,362
elements   3,787
with       2,007
similar    2,714
properties   5,144
.          1,012
no         2,053
one        2,028
could      2,071
come       2,272
up         2,039
with       2,007
a          1,037
good       2,204
solution   5,576
.          1,012
it         2,009
wasn       2,347
##t       

In [7]:
#first occurence of [SEP] token
sep_idx = input_ids.index(tokenizer.sep_token_id)
print("SEP token index: ", sep_idx)#number of tokens in segment A (question) - this will be one more than the sep_idx as the index in Python starts from 0
num_seg_a = sep_idx+1
print("Number of tokens in segment A: ", num_seg_a)#number of tokens in segment B (text)
num_seg_b = len(input_ids) - num_seg_a
print("Number of tokens in segment B: ", num_seg_b)#creating the segment ids
segment_ids = [0]*num_seg_a + [1]*num_seg_b#making sure that every input token has a segment id
assert len(segment_ids) == len(input_ids)

SEP token index:  14
Number of tokens in segment A:  15
Number of tokens in segment B:  222


In [8]:
#token input_ids to represent the input and token segment_ids to differentiate our segments - question and text
output = model(torch.tensor([input_ids]),  token_type_ids=torch.tensor([segment_ids]))

In [14]:
#tokens with highest start and end scores
answer_start = torch.argmax(output.start_logits)
answer_end = torch.argmax(output.end_logits)
if answer_end >= answer_start:
    answer = " ".join(tokens[answer_start:answer_end+1])
else:
    print("I am unable to find the answer to this question. Can you please ask another question?")
    
print("\nQuestion:\n{}".format(question.capitalize()))
print("\nAnswer:\n{}.".format(answer.capitalize()))


Question:
What team does he play for?

Answer:
6 . 1 . you can learn more about him and his work at this ur ##l : http : / / videos . how ##st ##uf ##f ##works . com / men ##del ##ee ##v developed the first periodic table of the elements in 1869 . he organized the elements by increasing atomic mass . he used his table to predict unknown elements . these were later discovered . the modern periodic table is based on atomic number . elements in each period go from metals on the left to metal ##loid ##s and then non ##met ##als on the right . within groups , elements have similar properties.


In [13]:
answer = tokens[answer_start]
for i in range(answer_start+1, answer_end+1):
    if tokens[i][0:2] == "##":
        answer += tokens[i][2:]
    else:
        answer += " " + tokens[i]

In [15]:
def question_answer(question, text):
    
    #tokenize question and text as a pair
    input_ids = tokenizer.encode(question, text)
    
    #string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    #segment IDs
    #first occurence of [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)    #number of tokens in segment A (question)
    num_seg_a = sep_idx+1    #number of tokens in segment B (text)
    num_seg_b = len(input_ids) - num_seg_a
    
    #list of 0s and 1s for segment embeddings
    segment_ids = [0]*num_seg_a + [1]*num_seg_b    
    assert len(segment_ids) == len(input_ids)
    
    #model output using input_ids and segment_ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    
    #reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    if answer_end >= answer_start:
      answer = tokens[answer_start]
      for i in range(answer_start+1, answer_end+1):
          if tokens[i][0:2] == "##":
              answer += tokens[i][2:]
          else:
              answer += " " + tokens[i]
                
    if answer.startswith("[CLS]"):
        answer = "Unable to find the answer to your question."
    
    print("\nPredicted answer:\n{}".format(answer.capitalize()))

In [16]:
text = input("Please enter your text: \n")
question = input("\nPlease enter your question: \n")
while True:
    question_answer(question, text)
    
    flag = True
    flag_N = False
    
    while flag:
        response = input("\nDo you want to ask another question based on this text (Y/N)? ")
        if response[0] == "Y":
            question = input("\nPlease enter your question: \n")
            flag = False
        elif response[0] == "N":
            print("\nBye!")
            flag = False
            flag_N = True
            
    if flag_N == True:
        break

Please enter your text: 
Virat Kohli is an Indian international cricketer and former captain of the India national cricket team. Widely regarded as one of the greatest batsmen of all time, Kohli plays as a right-handed batsman for Royal Challengers Bangalore in the Indian Premier League and for Delhi in domestic cricket.

Please enter your question: 
What team does he play for?

Predicted answer:
Royal challengers bangalore

Do you want to ask another question based on this text (Y/N)? N

Bye!
